In [134]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
import json

from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, LSTM, Embedding, SpatialDropout1D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D

In [135]:
df_idf = pd.read_csv('amazon/reviews.csv')
df_dataset = pd.read_json('clothing_dataset/renttherunway_final_data.json', lines = True)

In [136]:
def pre_process(text):
    # to lowercase
    text=text.lower()
    
    # remove tags
    text = re.sub("&lt;/?.*?&gt;", "&lt;&gt; ", text)
    
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)
    
    return text

def get_stop_words(stop_file_path):
    with open(stop_file_path, 'r', encoding='utf-8') as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

In [137]:
df_idf['text'] = df_idf['title'] + " " + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x: pre_process(str(x)))

df_dataset['text'] = df_dataset['review_summary'] + " " + df_dataset['review_text']
df_dataset['text'] = df_dataset['text'].apply(lambda x: pre_process(str(x)))

sub_dataset = df_dataset[['text', 'rating']]

In [138]:
all_data = df_idf['text'].append(sub_dataset['text'])
all_rating = df_idf['rating'].append(sub_dataset['rating'])

In [139]:
y = all_rating
y[:len(df_idf)] = y[:len(df_idf)].apply(lambda x: 1 if x > 3.5 else 0)#y.apply(lambda x: 1 if x > 3.5 else 0) 
y[len(df_idf):] = y[len(df_idf):].apply(lambda x: 1 if x > 5 else 0)#y.apply(lambda x: 1 if x > 3.5 else 0) 
y = y.to_numpy()

In [140]:
# Change this to use both datasets
#X_train, X_test, y_train, y_test = train_test_split(all_data, y, test_size=0.1, random_state=37)
# X_train = df_dataset['text']
# y_train = y[len(df_idf):]

# X_test = df_idf['text'].to_numpy()
# y_test = y[:len(df_idf)]

# print('# Train data samples:', X_train.shape)
# print('# Test data samples:', X_test.shape)

# print('Sample train', X_train[0])
# print('\nSample test', X_test[0])
# print(X_train.shape, y_train.shape)
# print(X_test.shape, y_test.shape)
# assert X_train.shape[0] == y_train.shape[0]
# assert X_test.shape[0] == y_test.shape[0]

In [141]:
MAX_LEN = 5260
GLOVE_DIM = 300
NB_WORDS = 49781

In [142]:
from keras.preprocessing.text import Tokenizer

tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               split=" ")
tk.fit_on_texts(all_data)

all_data_seq = tk.texts_to_sequences(all_data)

In [143]:
X_train_seq = all_data_seq[len(df_idf):]
X_test_seq = all_data_seq[:len(df_idf)]

y_train = y[len(df_idf):]
y_test = y[:len(df_idf)]

In [144]:
seq_lengths = all_data.apply(lambda x: len(x.split(' ')))
print(seq_lengths.describe())

count    275359.000000
mean         63.517557
std          69.966379
min           1.000000
25%          26.000000
50%          50.000000
75%          82.000000
max        5260.000000
Name: text, dtype: float64


In [145]:
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_test_seq_trunc = pad_sequences(X_test_seq, maxlen=MAX_LEN)

In [186]:
# # This is to split the X_test_seq_trunc in two normalize dataset for GANs
# print(X_test_seq_trunc.shape)

# positives = X_test_seq_trunc[0:1]
# negatives = X_test_seq_trunc[0:1]

# normalized_data = X_test_seq_trunc[0:1]

# print(positives.shape)

# for i in range(0, y_test.shape[0]):
#     if y_test[i] == 1:
#         positives = np.append(positives, X_test_seq_trunc[i:(i+1)], axis=0)
#     else:
#         negatives = np.append(negatives, X_test_seq_trunc[i:(i+1)], axis=0)
        
# iter_ = min(positives.shape[0], negatives.shape[0])
# for i in range(0, iter_):
#     normalized_data = np.append(normalized_data, positives[i:(i+1)], axis=0)
#     normalized_data = np.append(normalized_data, negatives[i:(i+1)], axis=0)
    
        
# print(positives.shape, negatives.shape, normalized_data.shape)

# for i in range(0, 5000):
#     if(positives[0:1, i] != 0):
#         print(positives[0:1, i], X_test_seq_trunc[0:1, i], i)

half_batch = int(128/2)

idx = np.random.randint(0, X_test_seq_trunc.shape[1], half_batch)
samp = np.array([])
count = 0
for i in range(0, half_batch):
    if count != half_batch/2:
        idx = np.random.randint(0, X_test_seq_trunc.shape[1], 1)
        while(y_test[idx] == 0):
            idx = np.random.randint(0, X_test_seq_trunc.shape[1], 1)
        samp = np.append(samp, idx, axis=0)
        
        while(y_test[idx] == 1):
            idx = np.random.randint(0, X_test_seq_trunc.shape[1], 1)
        samp = np.append(samp, idx, axis=0)
        count += 1
        


# # samp = X_test_seq_trunc[idx]
samp = samp.astype(int)

print(X_test_seq_trunc[samp])
print(y_test[samp])
print(samp)
df_idf['text'][samp]

[[    0     0     0 ...    84   204   520]
 [    0     0     0 ...     1   792 23123]
 [    0     0     0 ...  1027    16     5]
 ...
 [    0     0     0 ...     6    26   525]
 [    0     0     0 ...   177     1   982]
 [    0     0     0 ...    43    20   708]]
[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
[1316 2351 3203 3428  438 4732 1238 3755 4516 3807 1202 2107 2453  211
 5116  491 2304 4664 2475  203 2261 1930  933 2988 1155  286  881  818
 2526 3964 4557 1685  874 3797 4419  323  410 2009 4719 3970 3222 1220
 2900 5233 3726 2635 1843 5048 1681 5110 1290  925 5257 1959 1338 3823
 2905 3582 2938  349  209 3179 2263 3545]


1316    perfect for all my daily activities i loved th...
2351    not a good emergency only phone i activated th...
3203    flawless coming from another smartphone that w...
3428    i actually never imagined a phone could be thi...
438     excellent service responses excellent service ...
4732    absolutely hate it this phone could not be wor...
1238    so far so good have only had it for days but s...
3755    it happens every day now and it s annoying bec...
4516    five stars i already wrote a review they are g...
3807    think twice before buying there are definitely...
1202    had exactly what i needed when i needed it tha...
2107    no external memory can t save items this was m...
2453    i like this phone and i give it four stars out...
211     it s ok except for the browser button seems li...
5116    excellent bought the phone for my daughter rec...
491     make sure your aware that it can only be used ...
2304              i love it just what i needed i love it 
4664    one st

In [147]:
print(X_train_seq_trunc[10])  # Example of padded sequence
print(X_train_seq_trunc.shape)
print(X_test_seq_trunc.shape)
print(y_train.shape)
print(y_test.shape)

[  0   0   0 ...  61  14 316]
(192544, 5260)
(82815, 5260)
(192544,)
(82815,)


In [148]:
X_train_emb, X_valid_emb, y_train_emb, y_valid_emb = train_test_split(X_train_seq_trunc, y_train, test_size=0.1, random_state=37)

assert X_valid_emb.shape[0] == y_valid_emb.shape[0]
assert X_train_emb.shape[0] == y_train_emb.shape[0]

print('Shape of validation set:', X_valid_emb.shape)

Shape of validation set: (19255, 5260)


In [149]:
# This read the embeddings
# glove_file = 'glove.42B.' + str(GLOVE_DIM) + 'd.txt'
# emb_dict = {}
# glove = open(glove_file)
# for line in glove:
#     values = line.split()
#     word = values[0]
#     vector = np.asarray(values[1:], dtype='float32')
#     emb_dict[word] = vector
# glove.close()

In [150]:
airline_words = ['car', 'nice', 'flight', 'luggage']
for w in airline_words:
    if w in emb_dict.keys():
        print('Found the word {} in the dictionary'.format(w))
# print(emb_dict)

Found the word car in the dictionary
Found the word nice in the dictionary
Found the word flight in the dictionary
Found the word luggage in the dictionary


In [151]:
# Here we build a matrix that represent words and it corresponding emdg
emb_matrix = np.zeros((NB_WORDS, GLOVE_DIM))

for w, i in tk.word_index.items():
    if i < NB_WORDS:
        vect = emb_dict.get(w)
        if vect is not None:
            emb_matrix[i] = vect
    else:
        break

In [152]:
glove_model = Sequential()
glove_model.add(Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN))
# glove_model.add(LSTM(GLOVE_DIM, return_sequences=True))
# glove_model.add(LSTM(GLOVE_DIM, return_sequences=True))
glove_model.add(Flatten())
glove_model.add(Dense(1, activation='sigmoid'))
glove_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5260, 300)         14934300  
_________________________________________________________________
flatten_1 (Flatten)          (None, 1578000)           0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 1578001   
Total params: 16,512,301
Trainable params: 16,512,301
Non-trainable params: 0
_________________________________________________________________


In [153]:
glove_model.layers[0].set_weights([emb_matrix])
glove_model.layers[0].trainable = False

glove_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [154]:
history = glove_model.fit(X_train_emb
                       , y_train_emb
                       , epochs=5
                       , batch_size=32
                       , validation_data=(X_valid_emb, y_valid_emb)
                       , verbose=1)

Train on 173289 samples, validate on 19255 samples
Epoch 1/5
173289/173289 [==============================] - 637s 4ms/step - loss: 0.1121 - acc: 0.9751 - val_loss: 0.1034 - val_acc: 0.9747
Epoch 2/5
173289/173289 [==============================] - 680s 4ms/step - loss: 0.0806 - acc: 0.9791 - val_loss: 0.1231 - val_acc: 0.9727
Epoch 3/5
173289/173289 [==============================] - 628s 4ms/step - loss: 0.0675 - acc: 0.9819 - val_loss: 0.1317 - val_acc: 0.9774
Epoch 4/5
173289/173289 [==============================] - 658s 4ms/step - loss: 0.0589 - acc: 0.9838 - val_loss: 0.1307 - val_acc: 0.9760
Epoch 5/5
173289/173289 [==============================] - 585s 3ms/step - loss: 0.0524 - acc: 0.9851 - val_loss: 0.1415 - val_acc: 0.9708


In [155]:
print(glove_model.predict(X_train_emb[0:1]))
print(X_train_emb[0].shape)

[[0.9999999]]
(5260,)


# Adversarial Neural Network

In this section we start building the GANs, this model takes the word embedding and generate new embeddings that are similar to the given ones. 

In [156]:
def build_generator(shape):
    img_shape = shape
    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

def build_discriminator(shape):

    img_shape = shape

    model = Sequential()

#     model.add(Flatten(input_shape=img_shape)) # is one dimension
    model.add(Dense(512, input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)


In [157]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
def results(pred, actual):
    results = confusion_matrix(actual, pred)
    print('Confusion Matrix :')
    print(results)
    print ('Accuracy Score :',accuracy_score(actual, pred))
    print ('Report : ')
    print(classification_report(actual, pred))
    print()

In [158]:
img_rows = 1
img_cols = X_train_emb[0].shape
img_shape = (img_cols)

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build and compile the generator
generator = build_generator(img_shape)
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# The generator takes noise as input and generated imgs
z = Input(shape=(100,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The valid takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               2693632   
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 257       
Total params: 2,825,217
Trainable params: 2,825,217
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


In [159]:
def train(epochs, data, batch_size=128):


        # Rescale -1 to 1
        X_train = data #(X_train.astype(np.float32) - 127.5) / 127.5
#         X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
#             idx = np.random.randint(0, X_train.shape[1], half_batch)
#             imgs = X_train[idx]

            idx = np.random.randint(0, X_train.shape[1], half_batch)
            samp = np.array([])
            count = 0
            for i in range(0, half_batch):
                if count != half_batch/2:
                    idx = np.random.randint(0, X_train.shape[1], 1)
                    while(y_test[idx] == 0):
                        idx = np.random.randint(0, X_train.shape[1], 1)
                    samp = np.append(samp, idx, axis=0)

                    while(y_test[idx] == 1):
                        idx = np.random.randint(0, X_train.shape[1], 1)
                    samp = np.append(samp, idx, axis=0)
                    count += 1


            # samp = X_test_seq_trunc[idx]
            samp = samp.astype(int)
            imgs = X_train[samp]
#             print(np.sum(y_test[samp]))
            
            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = np.round(generator.predict(noise))

            # Train the discriminator
            d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))



In [160]:
train(epochs=4000, data=X_test_seq_trunc) # Generate data similar to the second dataset

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.810067, acc.: 36.72%] [G loss: 0.641288]
1 [D loss: 0.822508, acc.: 55.47%] [G loss: 0.539981]
2 [D loss: 0.704065, acc.: 53.91%] [G loss: 0.440695]
3 [D loss: 0.705125, acc.: 54.69%] [G loss: 0.373934]
4 [D loss: 0.812922, acc.: 51.56%] [G loss: 0.345146]
5 [D loss: 1.087976, acc.: 50.00%] [G loss: 0.308388]
6 [D loss: 0.958924, acc.: 50.78%] [G loss: 0.310668]
7 [D loss: 1.603963, acc.: 47.66%] [G loss: 0.300471]
8 [D loss: 1.038210, acc.: 49.22%] [G loss: 0.290471]
9 [D loss: 1.533416, acc.: 46.88%] [G loss: 0.290719]
10 [D loss: 1.701197, acc.: 46.88%] [G loss: 0.221435]
11 [D loss: 1.214371, acc.: 50.78%] [G loss: 0.238506]
12 [D loss: 1.186348, acc.: 50.78%] [G loss: 0.224354]
13 [D loss: 1.349773, acc.: 49.22%] [G loss: 0.267747]
14 [D loss: 0.933843, acc.: 50.78%] [G loss: 0.281795]
15 [D loss: 0.869056, acc.: 56.25%] [G loss: 0.306148]
16 [D loss: 1.054489, acc.: 53.12%] [G loss: 0.311180]
17 [D loss: 1.115238, acc.: 53.91%] [G loss: 0.314307]
18 [D loss: 0.981008

149 [D loss: 1.709611, acc.: 70.31%] [G loss: 0.947220]
150 [D loss: 0.672634, acc.: 71.09%] [G loss: 0.922792]
151 [D loss: 1.439794, acc.: 69.53%] [G loss: 0.871985]
152 [D loss: 1.623712, acc.: 56.25%] [G loss: 0.951732]
153 [D loss: 1.828401, acc.: 64.84%] [G loss: 0.968485]
154 [D loss: 2.022963, acc.: 64.84%] [G loss: 0.956028]
155 [D loss: 1.515982, acc.: 64.84%] [G loss: 0.916122]
156 [D loss: 1.453091, acc.: 71.88%] [G loss: 0.906584]
157 [D loss: 2.502814, acc.: 57.81%] [G loss: 0.657890]
158 [D loss: 1.270637, acc.: 59.38%] [G loss: 0.802096]
159 [D loss: 1.695183, acc.: 62.50%] [G loss: 0.919700]
160 [D loss: 1.172889, acc.: 67.19%] [G loss: 0.973992]
161 [D loss: 1.459300, acc.: 70.31%] [G loss: 0.979617]
162 [D loss: 1.608061, acc.: 64.06%] [G loss: 1.025523]
163 [D loss: 1.777737, acc.: 64.84%] [G loss: 0.963184]
164 [D loss: 1.943427, acc.: 65.62%] [G loss: 0.967020]
165 [D loss: 1.523937, acc.: 59.38%] [G loss: 0.947238]
166 [D loss: 1.295017, acc.: 63.28%] [G loss: 0.

296 [D loss: 0.714596, acc.: 89.06%] [G loss: 1.195927]
297 [D loss: 1.293511, acc.: 78.12%] [G loss: 1.234422]
298 [D loss: 1.510658, acc.: 80.47%] [G loss: 1.265035]
299 [D loss: 0.834029, acc.: 79.69%] [G loss: 1.246167]
300 [D loss: 0.905931, acc.: 82.81%] [G loss: 1.321465]
301 [D loss: 0.521523, acc.: 86.72%] [G loss: 1.316330]
302 [D loss: 1.369074, acc.: 81.25%] [G loss: 1.340075]
303 [D loss: 1.119299, acc.: 83.59%] [G loss: 1.299991]
304 [D loss: 1.496243, acc.: 82.81%] [G loss: 1.254269]
305 [D loss: 1.473108, acc.: 85.16%] [G loss: 1.294131]
306 [D loss: 1.363028, acc.: 85.16%] [G loss: 1.247320]
307 [D loss: 1.118316, acc.: 85.94%] [G loss: 1.321675]
308 [D loss: 1.407594, acc.: 75.00%] [G loss: 1.346804]
309 [D loss: 2.245696, acc.: 75.00%] [G loss: 1.397185]
310 [D loss: 0.618976, acc.: 86.72%] [G loss: 1.304172]
311 [D loss: 2.029838, acc.: 75.78%] [G loss: 1.323601]
312 [D loss: 1.461155, acc.: 86.72%] [G loss: 1.371235]
313 [D loss: 1.732625, acc.: 82.03%] [G loss: 1.

443 [D loss: 2.003702, acc.: 76.56%] [G loss: 1.568848]
444 [D loss: 1.629616, acc.: 76.56%] [G loss: 1.642401]
445 [D loss: 1.702112, acc.: 50.00%] [G loss: 0.364613]
446 [D loss: 1.462091, acc.: 50.78%] [G loss: 0.903962]
447 [D loss: 0.520578, acc.: 74.22%] [G loss: 1.923959]
448 [D loss: 0.290172, acc.: 90.62%] [G loss: 2.300140]
449 [D loss: 0.383202, acc.: 88.28%] [G loss: 1.984142]
450 [D loss: 0.265164, acc.: 89.06%] [G loss: 1.728876]
451 [D loss: 0.400966, acc.: 86.72%] [G loss: 1.469148]
452 [D loss: 0.474052, acc.: 82.81%] [G loss: 1.483725]
453 [D loss: 0.502260, acc.: 77.34%] [G loss: 1.540681]
454 [D loss: 0.325179, acc.: 85.94%] [G loss: 1.571933]
455 [D loss: 0.512734, acc.: 77.34%] [G loss: 1.402898]
456 [D loss: 0.580449, acc.: 75.00%] [G loss: 1.454484]
457 [D loss: 0.689765, acc.: 82.81%] [G loss: 1.508670]
458 [D loss: 1.205544, acc.: 80.47%] [G loss: 1.456156]
459 [D loss: 0.707895, acc.: 79.69%] [G loss: 1.387047]
460 [D loss: 0.694075, acc.: 78.12%] [G loss: 1.

...

3331 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3332 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3333 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3334 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3335 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3336 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3337 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3338 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3339 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3340 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3341 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3342 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3343 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3344 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3345 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3346 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3347 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.11809

3470 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3471 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3472 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3473 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3474 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3475 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3476 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3477 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3478 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3479 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3480 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3481 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3482 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3483 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3484 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3485 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3486 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.11809

3609 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3610 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3611 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3612 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3613 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3614 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3615 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3616 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3617 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3618 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3619 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3620 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3621 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3622 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3623 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3624 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3625 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.11809

3748 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3749 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3750 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3751 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3752 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3753 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3754 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3755 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3756 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3757 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3758 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3759 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3760 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3761 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3762 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3763 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3764 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.11809

3887 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3888 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3889 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3890 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3891 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3892 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3893 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3894 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3895 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3896 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3897 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3898 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3899 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3900 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3901 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3902 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118099]
3903 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.11809

# Model Evaluation

Second model trained on the real data

In [161]:
print(X_test_seq_trunc.shape, y_test.shape)

(82815, 5260) (82815,)


In [162]:
X_train_emb2, X_valid_emb2, y_train_emb2, y_valid_emb2 = train_test_split(X_test_seq_trunc, y_test, test_size=0.1, random_state=37)

assert X_valid_emb2.shape[0] == y_valid_emb2.shape[0]
assert X_train_emb2.shape[0] == y_train_emb2.shape[0]

print('Shape of validation set:',X_valid_emb2.shape)

Shape of validation set: (8282, 5260)


In [163]:
glove_model2 = Sequential()
glove_model2.add(Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN))
# glove_model2.add(LSTM(GLOVE_DIM, return_sequences=True))
# glove_model2.add(LSTM(GLOVE_DIM, return_sequences=True))
glove_model2.add(Flatten())
glove_model2.add(Dense(1, activation='sigmoid'))
glove_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 5260, 300)         14934300  
_________________________________________________________________
flatten_2 (Flatten)          (None, 1578000)           0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 1578001   
Total params: 16,512,301
Trainable params: 16,512,301
Non-trainable params: 0
_________________________________________________________________


In [164]:
glove_model2.layers[0].set_weights([emb_matrix])
glove_model2.layers[0].trainable = False

glove_model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [165]:
history2 = glove_model2.fit(X_train_emb2
                       , y_train_emb2
                       , epochs=5
                       , batch_size=32
                       , validation_data=(X_valid_emb2, y_valid_emb2)
                       , verbose=1)

Train on 74533 samples, validate on 8282 samples
Epoch 1/5
74533/74533 [==============================] - 304s 4ms/step - loss: 0.4588 - acc: 0.8406 - val_loss: 0.4640 - val_acc: 0.8582
Epoch 2/5
74533/74533 [==============================] - 293s 4ms/step - loss: 0.3073 - acc: 0.8952 - val_loss: 0.4720 - val_acc: 0.8654
Epoch 3/5
74533/74533 [==============================] - 286s 4ms/step - loss: 0.2581 - acc: 0.9113 - val_loss: 0.4751 - val_acc: 0.8666
Epoch 4/5
74533/74533 [==============================] - 292s 4ms/step - loss: 0.2322 - acc: 0.9201 - val_loss: 0.4734 - val_acc: 0.8723
Epoch 5/5
74533/74533 [==============================] - 279s 4ms/step - loss: 0.2104 - acc: 0.9277 - val_loss: 0.5164 - val_acc: 0.8708


In [166]:
# print(glove_model2.predict(X_train_emb[0:1]))
# print(X_train_emb[0].shape)

Third model trained on a generated dataset

In [167]:
# Generating data and labels
gen = 100000
noise = np.random.normal(0, 1, (gen, 100))
gen_samp = np.absolute((generator.predict(noise)))
prediction = glove_model.predict((gen_samp))
prediction = np.round(prediction)

In [168]:

# print(np.round(prediction[0:100]))
# print(np.sum(np.round(glove_model.predict(X_train_emb[0:100]))))
# print(np.sum(y_train[0:100]))


In [169]:
X_train_emb3, X_valid_emb3, y_train_emb3, y_valid_emb3 = train_test_split(gen_samp, prediction, test_size=0.3, random_state=37)

assert X_valid_emb3.shape[0] == y_valid_emb3.shape[0]
assert X_train_emb3.shape[0] == y_train_emb3.shape[0]

print('Shape of validation set:',X_valid_emb3.shape)

Shape of validation set: (30000, 5260)


In [170]:
glove_model3 = Sequential()
glove_model3.add(Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN))
# glove_model3.add(LSTM(GLOVE_DIM, return_sequences=True))
# glove_model3.add(LSTM(GLOVE_DIM, return_sequences=True))
glove_model3.add(Flatten())
glove_model3.add(Dense(1, activation='sigmoid'))
glove_model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5260, 300)         14934300  
_________________________________________________________________
flatten_3 (Flatten)          (None, 1578000)           0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 1578001   
Total params: 16,512,301
Trainable params: 16,512,301
Non-trainable params: 0
_________________________________________________________________


In [171]:
glove_model3.layers[0].set_weights([emb_matrix])
glove_model3.layers[0].trainable = False

glove_model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [172]:
history3 = glove_model3.fit(X_train_emb3
                       , y_train_emb3
                       , epochs=5
                       , batch_size=32
                       , validation_data=(X_valid_emb3, y_valid_emb3)
                       , verbose=1)

Train on 70000 samples, validate on 30000 samples
Epoch 1/5
70000/70000 [==============================] - 348s 5ms/step - loss: 0.2424 - acc: 0.9930 - val_loss: 0.1119 - val_acc: 0.9971
Epoch 2/5
70000/70000 [==============================] - 315s 5ms/step - loss: 0.0714 - acc: 0.9973 - val_loss: 0.0411 - val_acc: 0.9982
Epoch 3/5
70000/70000 [==============================] - 307s 4ms/step - loss: 0.0283 - acc: 0.9980 - val_loss: 0.0193 - val_acc: 0.9983
Epoch 4/5
70000/70000 [==============================] - 318s 5ms/step - loss: 0.0145 - acc: 0.9984 - val_loss: 0.0103 - val_acc: 0.9986
Epoch 5/5
70000/70000 [==============================] - 333s 5ms/step - loss: 0.0094 - acc: 0.9985 - val_loss: 0.0072 - val_acc: 0.9987


In this area I am going to compare their results

In [173]:
# The model trained on the generated dataset over the real dataset
actual = y_valid_emb3
pred = np.round(glove_model2.predict(X_valid_emb3))
pred2 = np.round(glove_model3.predict(X_valid_emb3))

In [174]:
results(pred, actual) # trained on original over real
results(pred2, actual) # generated

Confusion Matrix :
[[  337  1727]
 [ 5420 22516]]
Accuracy Score : 0.7617666666666667
Report : 
              precision    recall  f1-score   support

         0.0       0.06      0.16      0.09      2064
         1.0       0.93      0.81      0.86     27936

   micro avg       0.76      0.76      0.76     30000
   macro avg       0.49      0.48      0.47     30000
weighted avg       0.87      0.76      0.81     30000


Confusion Matrix :
[[ 2042    22]
 [   17 27919]]
Accuracy Score : 0.9987
Report : 
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      2064
         1.0       1.00      1.00      1.00     27936

   micro avg       1.00      1.00      1.00     30000
   macro avg       1.00      0.99      0.99     30000
weighted avg       1.00      1.00      1.00     30000




In [177]:
# The model trained on the generated dataset over the real dataset
actual = y_valid_emb2
pred = np.round(glove_model2.predict(X_valid_emb2))
pred2 = np.round(glove_model3.predict(X_valid_emb2))
pred3 = np.round(glove_model.predict(X_valid_emb2))


In [178]:
results(pred, actual) # trained on original over real
results(pred2, actual) # generated
results(pred3, actual) # original trained on the clothing

Confusion Matrix :
[[2112  596]
 [ 474 5100]]
Accuracy Score : 0.8708041535860903
Report : 
              precision    recall  f1-score   support

         0.0       0.82      0.78      0.80      2708
         1.0       0.90      0.91      0.91      5574

   micro avg       0.87      0.87      0.87      8282
   macro avg       0.86      0.85      0.85      8282
weighted avg       0.87      0.87      0.87      8282


Confusion Matrix :
[[  13 2695]
 [  21 5553]]
Accuracy Score : 0.6720598889157209
Report : 
              precision    recall  f1-score   support

         0.0       0.38      0.00      0.01      2708
         1.0       0.67      1.00      0.80      5574

   micro avg       0.67      0.67      0.67      8282
   macro avg       0.53      0.50      0.41      8282
weighted avg       0.58      0.67      0.54      8282


Confusion Matrix :
[[ 661 2047]
 [ 183 5391]]
Accuracy Score : 0.7307413668196088
Report : 
              precision    recall  f1-score   support

         0.0 